In [1]:
import uproot
import awkward as ak
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

from cfg.hnl_mva_tools import read_json_file
from data_tools.load_data import read_files_and_open_trees, filter_trees
from plot_tools.plot_vars_dist_tools import plot_var_dist, load_sig_data, load_bkg_data, plot_2d_var_dist

In [2]:
ntuples_json = "cfg/ntuples.json"
vars_json = "cfg/vars_new.json"
plots_dir = "../vars_dist_plots"
(
    sig_trees,
    bkg_trees,
    good_vars,
    sig_labels,
    bkg_labels
) = read_files_and_open_trees(ntuples_json, vars_json)

full_vars = read_json_file(vars_json)["vars"]
training_vars = read_json_file(vars_json)["training_vars"]
scale_factor_vars = read_json_file(vars_json)["scale_factors"]
weight_name = read_json_file(ntuples_json)["weight_name"]

# FILTER TREES
mass_list = ["mN1p0", "mN1p5"]
ctau_list = ["ctau10"]
my_sig_trees, my_sig_labels = filter_trees(
    sig_trees, sig_labels, mass_list=mass_list, ctau_list=ctau_list
)



In [3]:
#load_sig_data and load_bkg_data

bkg, bkg_weight = load_bkg_data(bkg_trees, training_vars,weight_name,scale_factor_vars)

Background Variables Loaded!


In [4]:
total = len(my_sig_trees) * len(training_vars)
progress = 0

for my_sig_tree, my_sig_label in zip(my_sig_trees, my_sig_labels):
    out_dir = f"{plots_dir}/{my_sig_label}"
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    sig, sig_weight = load_sig_data(my_sig_tree, training_vars, scale_factor_vars)
    for var1 in training_vars:
        # Update progress
        progress += 1
        print(f"Progress: [{'#' * (progress * 20 // total)}{' ' * (20 - progress * 20 // total)}] {progress}/{total}")

        plot_var_dist(sig,bkg,sig_weight,bkg_weight,my_sig_label,bkg_labels,var1,out_dir)
        # for var2 in training_vars:
        #     if var1 == var2:
        #         continue
        #     plot_2d_var_dist(sig,bkg,sig_weight,bkg_weight,var1,var2,out_dir)

Loading Signal Variables...
Signal Variables Loaded!
Progress: [                    ] 1/48
Progress: [                    ] 2/48
Progress: [#                   ] 3/48
Progress: [#                   ] 4/48
Progress: [##                  ] 5/48
Progress: [##                  ] 6/48
Progress: [##                  ] 7/48
Progress: [###                 ] 8/48
Progress: [###                 ] 9/48
Progress: [####                ] 10/48
Progress: [####                ] 11/48
Progress: [#####               ] 12/48
Progress: [#####               ] 13/48
Progress: [#####               ] 14/48
Progress: [######              ] 15/48
Progress: [######              ] 16/48
Progress: [#######             ] 17/48
Progress: [#######             ] 18/48
Progress: [#######             ] 19/48
Progress: [########            ] 20/48
Progress: [########            ] 21/48
Progress: [#########           ] 22/48
Progress: [#########           ] 23/48
Progress: [##########          ] 24/48
Loading Signal Varia